<b> Final project <b>
    
    
Last May for my R project, I studied russian verbs (prygnul) and verboids (pryg) on the basis of Russian Corpus. I investigated what pattern can affect the choice of a verboid instead of a verb. I check three parameters: unexpectedness (The hypothesis was that verboids are used more often in the unxpected contexts), being part of a coordinated chain (The hypothesis was that verboids are used more often in the coordinated chains, cf.: 'on pryg i ybezhal') and the year of publication (The hypothesis was that verboids should be used in older texts). The first hypothesis was confirmed, the second one wasn't, but my data concerning the year of publication proved to be not very good because it seems that Russian corpus give a very bad random sample (not random at all concering the year of publication). 
Thus, for my final python project I decided to write a program that would extract all the examples (from all the search pages) fitting the inquiry so that I could then randomize it.
So as an input I give the link (to the any page except for the first one because it has a different default structure), the number of page and the name of the output file.
As an output I take the file with three columns: the text of the examples, the info about the auther and the date, the date.

In [141]:
import re
import urllib.request
import pandas as pd 
from tqdm import tqdm

Here is the code of the program for the fixed inquiry to check if it works

In [170]:
texts = []
info = []
dates = []
#search all the pages
for page in tqdm(range(3)):
    new_examples = []
    info_local = []
    #open the link
    req = urllib.request.Request('https://processing.ruscorpora.ru/search.xml?spp=50&text=lexgramm&level1=0&level2=0&api=1.0&spd=100&sem-mod2=sem&sem-mod2=sem2&sem-mod1=sem&sem-mod1=sem2&env=alpha&nodia=1&parent2=0&sort=i_grtagging&startyear=1800&min2=1&lang=ru&lex1=%D0%BF%D1%80%D1%8B%D0%B3&dpp=50&max2=1&endyear=2019&mode=main&parent1=0&p=' + str(page))
    with urllib.request.urlopen(req) as response:
        html = response.read().decode('utf-8')
    #extract the examples
    regEx = re.compile('<li>.*?</span> <span class="off"> ', flags= re.DOTALL)
    examples = regEx.findall(html)
    #delete rubish
    regTitle = re.compile(".*?Все примеры", re.DOTALL)
    regTag = re.compile('<.*?>', re.DOTALL)
    for ex in examples:
        clean_ex = regTag.sub("", ex)
        clean_ex = regTitle.sub("",clean_ex)
        clean_ex = clean_ex.replace("  ", " ")
        new_examples.append(clean_ex)
    #split the example into text and info about the publication
    for ex in new_examples:
        new_ex = ex.split("[")
        text = new_ex[0]
        #to be sure that phrases like "на том [берегу]" are not parsed like "info"
        for n in new_ex[1:]:
            res = re.search("\d\d\d\d", n)
            if res:
            
                info.append(n)
                #to work with it later
                info_local.append(n)
            else:
                text = new_ex[0] + "["  + n
        texts.append(text) 
    #extract information about the date from the "info"
    for i in range(len(info_local)):
        res = re.search("\d\d\d\d-\d\d\d\d", info_local[i])
        if not res:
            res = re.search("\d\d\d\d", info_local[i])
        dates.append(res.group())
#make a table
pryg = pd.DataFrame({
    "texts": texts ,
    "info": info,
    "dates": dates,
})
pryg.to_csv("pryg.csv", encoding='utf-8-sig')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.67s/it]


,texts,info,dates
0,"А он прыг на форточку, а там четырнадцатый этаж.","Сергей Шикера. Египетское метро // «Волга», 20...",2016
1,Она с тумбочки прыг и покорно идет.,Сергей Носов. Фигурные скобки (2015)],2015
2,"Через игру научила командам «лежать», «сидеть»...",Диана Злобина. Зачем ученые лис приручили // «...,2014
3,Девочки уже раздали автографы и явно скучали ―...,Виктор Пелевин. S.N.U.F.F (2011)],2011
4,"― Пупарас Трыг ― мое сердце прыг, ― пробормот...",Виктор Пелевин. S.N.U.F.F (2011)],2011
...,...,...,...
191,"Одинъ разсказываетъ: «Иду я гдѣ-то, а мимо мен...",П. А. Ровинский. Черты из боевой жизни Черного...,1880
192,"Благословясь да богу помолясь, а не так как-ни...",М. Е. Салтыков-Щедрин. Господа Головлевы (1875...,1875-1880
193,все-то у тебя на уме прыг да шмыг!,М. Е. Салтыков-Щедрин. Господа Головлевы (1875...,1875-1880
194,Вот поравнялись мы с телегой… вдруг великан в ...,И. С. Тургенев. Стучит! (1874)],1874


the same but as function.
(in case I forget how it works)

In [172]:
import re
import urllib.request
import pandas as pd 
from tqdm import tqdm
def RussianCorpusExctractor(number_of_pages, output_file_name, url):
    texts = []
    info = []
    dates = []
    for page in tqdm(range(number_of_pages-1)):
        new_examples = []
        info_local = []
        req = urllib.request.Request(url[:-1]+str(page))
        with urllib.request.urlopen(req) as response:
            html = response.read().decode('utf-8')
        regEx = re.compile('<li>.*?</span> <span class="off"> ', flags= re.DOTALL)
        examples = regEx.findall(html)
        regTitle = re.compile(".*?Все примеры", re.DOTALL)
        regTag = re.compile('<.*?>', re.DOTALL)
        for ex in examples:
            clean_ex = regTag.sub("", ex)
            clean_ex = regTitle.sub("",clean_ex)
            clean_ex = clean_ex.replace("  ", " ")
            new_examples.append(clean_ex)
        for ex in new_examples:
            new_ex = ex.split("[")
            text = new_ex[0]
            for n in new_ex[1:]:
                res = re.search("\d\d\d\d", n)
                if res:
                    info.append(n)
                    info_local.append(n)
                else:
                    text = new_ex[0] + "["  + n
            texts.append(text)   
        for i in range(len(info_local)):
            res = re.search("\d\d\d\d-\d\d\d\d", info_local[i])
            if not res:
                res = re.search("\d\d\d\d", info_local[i])
            dates.append(res.group())
    data = pd.DataFrame({
        "texts": texts ,
        "info": info,
        "dates": dates,
    })
    return data.to_csv(output_file_name+".csv", encoding='utf-8-sig')

In [173]:
RussianCorpusExctractor(32, "prygnul", 'https://processing.ruscorpora.ru/search.xml?text=lexgramm&level1=0&level2=0&api=1.0&spd=50&out=normal&sem-mod2=sem&sem-mod2=sem2&sem-mod1=sem&sem-mod1=sem2&env=alpha&nodia=1&mode=main&sort=random&startyear=1800&min2=1&seed=9336&lang=ru&lex1=%22%D0%BF%D1%80%D1%8B%D0%B3%D0%BD%D1%83%D0%BB%22%7C%22%D0%BF%D1%80%D1%8B%D0%B3%D0%BD%D1%83%D0%BB%D0%B0%22%7C%22%D0%BF%D1%80%D1%8B%D0%B3%D0%BD%D1%83%D0%BB%D0%BE%22%7C%22%D0%BF%D1%80%D1%8B%D0%B3%D0%BD%D1%83%D0%BB%D0%B8%22&dpp=50&max2=1&endyear=2019&parent2=0&parent1=0&p=0')

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [02:12<00:00,  4.26s/it]
